In [1]:
import pandas as pd
import requests
import json
from pymongo import MongoClient
import geopandas
conn = MongoClient("localhost:27017")
db = conn.get_database("ironhack")
from pymongo import GEOSPHERE
import src.geoqueries as g
import src.maps as m
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
df = pd.read_csv("data/df.csv")

In [3]:
df = df.dropna()
df.reset_index(drop=True,inplace=True)
df.drop_duplicates(inplace=True)

In [4]:
db.preferences.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [5]:
collection = db.get_collection("preferences")

In [6]:
collection.find_one({})

{'_id': ObjectId('607bfae4b913636b15a4fb88'),
 'name': 'Madison Square Garden',
 'location': {'type': 'Point', 'coordinates': [-73.9935421944, 40.7507519651]},
 'place': 'basket'}

In [7]:
df.sample(3)

,company,city,latitude,longitude,zip_code
23,Yammer,San Francisco,37.776246,-122.417922,94103
39,Make It Rain,London,51.462600,-0.139082,SW4 0BG
7,Phreesia,New York,40.739794,-73.985878,10010


In [8]:
lat = list(df.latitude)
long = list(df.longitude)
coordinates = []

In [9]:
for lt,lg in zip(lat,long):
    coord = [lg,lt]
    coordinates.append({"type": "Point",  "coordinates": coord})

In [10]:
coordinates[6]

{'type': 'Point', 'coordinates': [-0.107843, 51.523676]}

In [11]:
df['geometry'] = coordinates

In [12]:
df.head()

,company,city,latitude,longitude,zip_code,geometry
0,Digg,San Francisco,37.764726,-122.394523,94107,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Powerset,San Francisco,37.778613,-122.395289,94107,"{'type': 'Point', 'coordinates': [-122.395289,..."
2,CastTV,San Francisco,37.780716,-122.393913,94107,"{'type': 'Point', 'coordinates': [-122.393913,..."
3,Grockit,San Francisco,37.775196,-122.419204,94107,"{'type': 'Point', 'coordinates': [-122.419204,..."
4,Dropbox,San Francisco,37.790943,-122.408499,94107,"{'type': 'Point', 'coordinates': [-122.4084994..."


### prueba

In [13]:
party_counts = []
starbucks_counts = []
school_counts = []
vegan_counts = []
basket_counts = []
for c in coordinates:
    query = {"location": {"$near": {"$geometry": c, "$maxDistance": 2000}}}
    list_ = list(collection.find(query))
    party = 0
    starbucks = 0
    school = 0
    vegan = 0
    basket = 0
    for l in list_:
        if l['place'] == 'party':
            party += 1
        elif l['place'] == 'starbucks':
            starbucks += 1
        elif l['place'] == 'school':
            school += 1
        elif l['place'] == 'vegan':
            vegan += 1
        elif l['place'] == 'basket':
            basket += 1
    party_counts.append(party)
    starbucks_counts.append(starbucks)
    school_counts.append(school)
    vegan_counts.append(vegan)
    basket_counts.append(basket)

In [14]:
df['party'] = party_counts
df['starbucks'] = starbucks_counts
df['school'] = school_counts
df['vegan'] = vegan_counts
df['basket'] = basket_counts

In [15]:
df.head()

,company,city,latitude,longitude,zip_code,geometry,party,starbucks,school,vegan,basket
0,Digg,San Francisco,37.764726,-122.394523,94107,"{'type': 'Point', 'coordinates': [-122.394523,...",7,4,0,1,3
1,Powerset,San Francisco,37.778613,-122.395289,94107,"{'type': 'Point', 'coordinates': [-122.395289,...",7,14,2,3,3
2,CastTV,San Francisco,37.780716,-122.393913,94107,"{'type': 'Point', 'coordinates': [-122.393913,...",6,13,0,0,3
3,Grockit,San Francisco,37.775196,-122.419204,94107,"{'type': 'Point', 'coordinates': [-122.419204,...",30,15,28,25,0
4,Dropbox,San Francisco,37.790943,-122.408499,94107,"{'type': 'Point', 'coordinates': [-122.4084994...",8,14,7,5,0


In [16]:
df_loc = df.drop(['zip_code'],axis=1)

In [17]:
df_loc.head()

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket
0,Digg,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",7,4,0,1,3
1,Powerset,San Francisco,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,...",7,14,2,3,3
2,CastTV,San Francisco,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,...",6,13,0,0,3
3,Grockit,San Francisco,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",30,15,28,25,0
4,Dropbox,San Francisco,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994...",8,14,7,5,0


### WEIGHT

In [18]:
weight_party = 0.25
weight_starbucks = 0.3
weight_basket = 0.05
weight_school = 0.25
weight_vegan = 0.15

### GRADES

In [19]:
max_ = df.party.max()
mean = df.party.mean()
print(max_, mean)

30 6.416666666666667


**PARTY GRADES**
    
    [0] : 0  
    [1-8] : 1  
    [8-20] : 2
    [20-30] : 3

In [20]:
max_ = df.starbucks.max()
mean = df.starbucks.mean()
print(max_, mean)

23 8.020833333333334


**STARBUCKS GRADES**
    
    [0] : 0  
    [1-8] : 1  
    [8-20] : 2
    [20-23] : 3

In [21]:
max_ = df.school.max()
mean = df.school.mean()
print(max_, mean)

28 5.270833333333333


**SCHOOL GRADES**
    
    [0] : 0  
    [1-8] : 1  
    [8-20] : 2
    [20-28] : 3

In [22]:
max_ = df.vegan.max()
mean = df.vegan.mean()
print(max_, mean)

29 5.166666666666667


**VEGAN GRADES**
    
    [0] : 0  
    [1-8] : 1  
    [8-20] : 2
    [20-29] : 3

**BASCKET GRADES**
    
    [0] : 0  
    [2] : 1  
    [3] : 2
    [4] : 3

In [23]:
df_loc['party_grades'] = df_loc["party"].apply(g.grades)
df_loc['starbucks_grades'] = df_loc["starbucks"].apply(g.grades)
df_loc['school_grades'] = df_loc["school"].apply(g.grades)
df_loc['vegan_grades'] = df_loc["vegan"].apply(g.grades)
df_loc['basket_grades'] = df_loc["basket"]

In [24]:
df_loc['weighted_party_grades'] = df_loc.apply(lambda row: row['party_grades']*weight_party, axis = 1)
df_loc['weighted_starbucks_grades'] = df_loc.apply(lambda row: row['starbucks_grades']*weight_party, axis = 1)
df_loc['weighted_school_grades'] = df_loc.apply(lambda row: row['school_grades']*weight_party, axis = 1)
df_loc['weighted_vegan_grades'] = df_loc.apply(lambda row: row['vegan_grades']*weight_party, axis = 1)
df_loc['weighted_basket_grades'] = df_loc.apply(lambda row: row['basket_grades']*weight_party, axis = 1)

In [25]:
df_loc['weighted_result'] =  df_loc.apply(lambda row: row['weighted_party_grades'] 
                                          + row['weighted_basket_grades']
                                          + row['weighted_starbucks_grades']
                                          + row['weighted_school_grades'] 
                                          + row['weighted_vegan_grades'],
                                          axis = 1)

In [26]:
df_loc.head()

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket,...,starbucks_grades,school_grades,vegan_grades,basket_grades,weighted_party_grades,weighted_starbucks_grades,weighted_school_grades,weighted_vegan_grades,weighted_basket_grades,weighted_result
0,Digg,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",7,4,0,1,3,...,1,0,1,3,0.25,0.25,0.00,0.25,0.75,1.50
1,Powerset,San Francisco,37.778613,-122.395289,"{'type': 'Point', 'coordinates': [-122.395289,...",7,14,2,3,3,...,2,1,1,3,0.25,0.50,0.25,0.25,0.75,2.00
2,CastTV,San Francisco,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,...",6,13,0,0,3,...,2,0,0,3,0.25,0.50,0.00,0.00,0.75,1.50
3,Grockit,San Francisco,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,...",30,15,28,25,0,...,2,3,3,0,0.75,0.50,0.75,0.75,0.00,2.75
4,Dropbox,San Francisco,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994...",8,14,7,5,0,...,2,1,1,0,0.25,0.50,0.25,0.25,0.00,1.25


In [27]:
df_loc.weighted_result.max()

3.0

In [28]:
df_ = [df_loc['weighted_result'] == df_loc.weighted_result.max()]

In [29]:
is_max = df_loc.loc[:, "weighted_result"] == df_loc.weighted_result.max()
df_final = df_loc.loc[is_max]

In [30]:
df_final

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket,...,starbucks_grades,school_grades,vegan_grades,basket_grades,weighted_party_grades,weighted_starbucks_grades,weighted_school_grades,weighted_vegan_grades,weighted_basket_grades,weighted_result
33,Box UK,London,51.510774,-0.139245,"{'type': 'Point', 'coordinates': [-0.1392447, ...",27,23,24,26,0,...,3,3,3,0,0.75,0.75,0.75,0.75,0.0,3.0


In [31]:
coordinates_ = [list(df_final['latitude'])[0],list(df_final['longitude'])[0]]

In [32]:
coordinates_

[51.5107737, -0.1392447]

In [33]:
london = {'type': 'Point', 'coordinates': [51.509865, -0.118092]}
map_lnd = Map(location=london['coordinates'],zoom_start=15)

In [34]:
icono = Icon(color = "white",
             prefix = "fa",
             icon = "trophy",
             icon_color = "black")

loc = {"location": coordinates_,
      "tooltip": "Gaming Company"}

In [35]:
marker_company = Marker(**loc, icon = icono)

In [36]:
marker_company.add_to(map_lnd)
map_lnd

In [37]:
map_lnd.save('decision')

In [38]:
df_final

,company,city,latitude,longitude,geometry,party,starbucks,school,vegan,basket,...,starbucks_grades,school_grades,vegan_grades,basket_grades,weighted_party_grades,weighted_starbucks_grades,weighted_school_grades,weighted_vegan_grades,weighted_basket_grades,weighted_result
33,Box UK,London,51.510774,-0.139245,"{'type': 'Point', 'coordinates': [-0.1392447, ...",27,23,24,26,0,...,3,3,3,0,0.75,0.75,0.75,0.75,0.0,3.0


In [39]:
party_coord = []
starbucks_coord = []
school_coord = []
vegan_coord = []
basket_coord = []

query = {"location": {"$near": {"$geometry": list(df_final['geometry'])[0], "$maxDistance": 2000}}}
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-0.1392447, 51.5107737]},
   '$maxDistance': 2000}}}

In [40]:
df_places = pd.DataFrame(list(collection.find(query)))

In [41]:
coordinates = []
for point in list(df_places['location']):
    coordinates.append([point['coordinates'][1],point['coordinates'][0]])
df_places['coordinates'] = coordinates

In [42]:
df_places.head()

,_id,name,location,place,coordinates
0,607bfb26b913636b15a4fc4c,Starbucks,"{'type': 'Point', 'coordinates': [-0.139154, 5...",starbucks,"[51.510288, -0.139154]"
1,607bfaf6b913636b15a4fba5,The Cuckoo Club,"{'type': 'Point', 'coordinates': [-0.137770716...",party,"[51.5099619985, -0.1377707164]"
2,607bfaf6b913636b15a4fb98,Tramp,"{'type': 'Point', 'coordinates': [-0.137835548...",party,"[51.5083318933, -0.1378355488]"
3,607bfb0db913636b15a4fc06,Malvern House Picadilly Centre,"{'type': 'Point', 'coordinates': [-0.134370783...",school,"[51.5101933333, -0.1343707833]"
4,607bfb26b913636b15a4fc4f,Starbucks,"{'type': 'Point', 'coordinates': [-0.139378, 5...",starbucks,"[51.513952, -0.139378]"


In [43]:

for i, row in df_places.iterrows():
    if row['place'] == 'starbucks':
        p = {
        "location": row["coordinates"],
        "tooltip" : row["name"]
        }
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "coffee",
                    icon_color = "black"
        )
    elif row['place'] == 'school':
        p = {
        "location": row["coordinates"],
        "tooltip" : row["name"]
        }
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "book",
                    icon_color = "black"
        )
    elif row['place'] == 'party':
        p = {
        "location": row["coordinates"],
        "tooltip" : row["name"]
        }
        icon = Icon(color = "pink",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
    elif row['place'] == 'vegan':
        p = {
        "location": row["coordinates"],
        "tooltip" : row["name"]
        }
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "cutlery",
                    icon_color = "black"
        )
    else:
        p = {
        "location": row["coordinates"],
        "tooltip" : row["name"]
        }
        icon = Icon(color = "orange",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "black"
        )
    Marker(**p,icon = icon ).add_to(map_lnd)
map_lnd        
        

In [44]:
df_places.drop(['_id'], axis = 1, inplace = True)

In [45]:
df_places.head()

,name,location,place,coordinates
0,Starbucks,"{'type': 'Point', 'coordinates': [-0.139154, 5...",starbucks,"[51.510288, -0.139154]"
1,The Cuckoo Club,"{'type': 'Point', 'coordinates': [-0.137770716...",party,"[51.5099619985, -0.1377707164]"
2,Tramp,"{'type': 'Point', 'coordinates': [-0.137835548...",party,"[51.5083318933, -0.1378355488]"
3,Malvern House Picadilly Centre,"{'type': 'Point', 'coordinates': [-0.134370783...",school,"[51.5101933333, -0.1343707833]"
4,Starbucks,"{'type': 'Point', 'coordinates': [-0.139378, 5...",starbucks,"[51.513952, -0.139378]"


In [46]:
list(df_final.geometry)[0]

{'type': 'Point', 'coordinates': [-0.1392447, 51.5107737]}

In [47]:
df_final.head()
new_row = {'name':'Gaming Company', 'location': list(df_final.geometry)[0], 'place' : 'company', 'coordinates' : [list(df_final.geometry)[0]['coordinates'][1], list(df_final.geometry)[0]['coordinates'][0]]}
df_places = df_places.append(new_row, ignore_index=True)

In [48]:
df_places.tail()

,name,location,place,coordinates
96,Academy Class,"{'type': 'Point', 'coordinates': [-0.116223596...",school,"[51.5028805129, -0.1162235969]"
97,Starbucks,"{'type': 'Point', 'coordinates': [-0.123353, 5...",starbucks,"[51.52392, -0.123353]"
98,London Waterloo Academy,"{'type': 'Point', 'coordinates': [-0.115453, 5...",school,"[51.503339, -0.115453]"
99,King's College London - Waterloo Campus,"{'type': 'Point', 'coordinates': [-0.112166203...",school,"[51.5055975107, -0.1121662031]"
100,Gaming Company,"{'type': 'Point', 'coordinates': [-0.1392447, ...",company,"[51.5107737, -0.1392447]"


In [52]:
latit = []
longit = []
for coord in list(df_places['coordinates']):
    latit.append(coord[0])
    longit.append(coord[1])

In [55]:
df_places['latitude'] = latit
df_places['longitude'] = longit

In [56]:
df_places.to_csv('data/places.csv')